In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from scipy.stats import mode

data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

def get_kmeans_accuracy(X_data, y_true):
    # 1. สร้างโมเดล K-Means (กำหนด 3 กลุ่มตามจำนวนสายพันธุ์ Iris)
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_data)

    # 2. สร้าง Array เปล่าไว้เก็บป้ายชื่อใหม่ที่แก้ไขแล้ว
    labels_mapped = np.zeros_like(clusters)

    # 3. วนลูปเช็คทีละกลุ่ม (0, 1, 2) เพื่อดูว่ากลุ่มนี้ตรงกับเฉลยอะไรมากที่สุด
    for i in range(3):
        mask = (clusters == i)
        # หาว่าในคลัสเตอร์ i นี้ ข้อมูลส่วนใหญ่คือสายพันธุ์อะไร (Mode)
        if np.sum(mask) > 0:
            labels_mapped[mask] = mode(y_true[mask], keepdims=True)[0][0]

    # 4. วัดความแม่นยำ
    return accuracy_score(y_true, labels_mapped)


# --- Case A: Full Features ---
acc_full = get_kmeans_accuracy(X_scaled, y)

# --- Case B: Cut 1 Feature ---
selector_1 = VarianceThreshold(threshold=0.035)
X_cut1 = selector_1.fit_transform(X_scaled)
acc_cut1 = get_kmeans_accuracy(X_cut1, y)

# --- Case C: Cut 2 Features ---
selector_2 = VarianceThreshold(threshold=0.06)
X_cut2 = selector_2.fit_transform(X_scaled)
acc_cut2 = get_kmeans_accuracy(X_cut2, y)


results = {
    "Experiment": ["Full Features", "Cut 1 Feature (Th 0.035)", "Cut 2 Features (Th 0.06)"],
    "Features Remaining": [X_scaled.shape[1], X_cut1.shape[1], X_cut2.shape[1]],
    "Accuracy": [acc_full, acc_cut1, acc_cut2]
}

df_compare = pd.DataFrame(results)
pd.options.display.float_format = '{:.4f}'.format

print("--- ผลลัพธ์การเปรียบเทียบ (Unsupervised: K-Means) ---")
print(df_compare.to_string(index=False))

--- ผลลัพธ์การเปรียบเทียบ (Unsupervised: K-Means) ---
              Experiment  Features Remaining  Accuracy
           Full Features                   4    0.8867
Cut 1 Feature (Th 0.035)                   3    0.9000
Cut 2 Features (Th 0.06)                   2    0.9600
